In [106]:
import numpy as np
from docplex.mp import model
from numba import njit
import math

# auxilary functions

We use numba decor to accelarate the computation

In [142]:
@njit
def combination(n: int, k: int) -> int:
    if k > n:
        return 0

    numerator = np.float64(1)
    denominator = np.float64(1)

    for i in range(k):
        numerator *= (n - i)
        denominator *= (i + 1)
    
    return numerator // denominator


class AuxFunc():

    @njit
    def prob_k(pimax: int, pij: float) -> np.array:
        """Generate probability.

        Args:
            pimax: int - maximum number of vehicles
            pij: float in [0, 1] - parameter for distribution

        Returns:
            float in (0, 1) - probability at time j, there are k vehicles of type i required
        """
        assert 0 <= pij <= 1
        
        prob = np.zeros(pimax)
        for k in range(pimax):
            #prob[k] = comb(pimax, k) * pij**k * (1-pij)**(pimax - k)
            prob[k] = combination(pimax,k) * pij**k * (1-pij)**(pimax - k)

        return prob

    @njit
    def get_prob(i: int, m: int, pimax: int, rho: np.array) -> np.array:
        """Generate probability.

        Args:
            n: int - number of vehicle types
            m: int - number of time periods
            pmax: np.array - maximum number of vehicles of each type
            rho: np.array(float) in [0,1] - parameter for distribution

        Returns:
            np.array - probability at time j, there are k vehicles of type i required
        """

        probi = np.zeros((m,pimax))
        for j in range(m):
            for k in range(pimax):
                probi[j,k] = abs(combination(pimax,k) * (rho[i,j]**k) * ((1-rho[i,j])**(pimax - k)))
            

        return probi

    @njit
    def get_value_fi(beta_i:float, gamma_i: float, prob_i: np.array, pimax: float, m: int, xi: float, i: int) -> float:
        """Generate cost value of type i

        Args:
            beta_i:float, 
            gamma_i: float, 
            prob_i: np.array, 
            pimax: float, 
            m: int
            x: current selection
            i: vehicle type i

        Returns:
            float: cost of maintenance and hiring more vehicles
        """
        # get cost for each stage j
        value_j = np.array([np.sum(np.array([ prob_i[j,k] 
                                            *(beta_i * min(k, xi) 
                                            + gamma_i * max(k - xi, 0)) 
                                            for k in range(pimax)]))
                                            for j in range(m)])

                
        return np.sum(value_j)


# Data generation


In [134]:
class DataGeneration:
    """Args:
    n: int - number of vehicle types
    m: int - number of time periods
    pm: int - max number of vehicles for total
    """
    def __init__(self, n: int, m: int, pm: int, random_state: int) -> None:

        self.n = n
        self.m = m
        self.pm = pm
        self.random_state = random_state

    def get_parameters(self):
        """Generate parameters.

        Returns:
            alpha: list of random integers between [1, 1000]
            beta: list of random integers between [1, 1000]
            gamma: list of random integers between [alpha[i]+beta[i], 2*(alpha[i]+beta[i])]
            pmax: list of maximum number of vehicles of each type are randon integer between [1,pm]
            rho: n x m array of random floats between [0, 1]
            c: list of random integers between [1, 100] (cost of each type of vehicle)
            b: capacity
        """
        np.random.seed(self.random_state)
        alpha = np.random.randint(2, 100, size=self.n)
        beta = np.random.randint(2, 100, size=self.n)
        gamma = np.random.randint((alpha + beta)+1, 2 * (alpha + beta))
        pmax = np.random.randint(int(self.pm/2), self.pm , size=self.n)
        rho = np.round(np.random.rand(self.n, self.m),2)
        c = np.random.randint(2, 10, size=self.n)
        b_min = int(np.sum(c*pmax)/self.n)
        b = np.random.randint(b_min, int(2*b_min))

        return alpha, beta, gamma, pmax, rho, c, b
    

# Algorithms design
1. standard cutting plane approach
2. linearization approach

In [170]:
class Algorithms:
    """Args:
    data: input data for the models
    timeout: time limits for each models, default 200s
    """
    def __init__(self, data, timeout: float = 200) -> None:
        paramter = data.get_parameters()
        rho = paramter[4]
        pmax = paramter[3]
        self.data = data
        self.m = data.m
        self.n = data.n
        self.pmax = pmax
        self.c = paramter[5]
        self.b = paramter[6]
        self.alpha = paramter[0]
        self.beta = paramter[1]
        self.gamma = paramter[2]
        self.rho = rho
        self.timeout = timeout
        self.prob = [AuxFunc.get_prob(i,data.m,pmax[i],rho) for i in range(self.n)]

    def get_value_fi(self, xi: float, i: int) -> float:
        """Generate cost value of type i

        Args:
            x: current selection
            i: vehicle type i

        Returns:
            float: cost of maintenance and hiring more vehicles
        """
                
        return AuxFunc.get_value_fi(self.beta[i],self.gamma[i],self.prob[i],self.pmax[i],self.m,xi,i)
    
    def get_value(self, x: np.array) -> float:
        """Generate cost value

        Args:
            x: current selection

        Returns:
            float: cost of maintenance and hiring more vehicles
        """
        
        return np.sum(np.array([self.get_value_fi(x[i], i)
                        for i in range(self.n)]))

    def get_subgrad_i(self, xi: float, i: int) -> float:
        """Generate subgradient for fi

        Args:
            i: index
            xi: current selection

        Returns:
            float: subgrad of fi at xi
        """

        return self.get_value_fi(xi+1,i) - self.get_value_fi(xi,i)

    def get_sub_gradient(self, x: np.array) -> np.array:
        """Generate subgradient

        Args:
            x: current selection

        Returns:
            np.array: an array of subgradient vector at x
        """
        subgrad = np.zeros(self.n)
        for i in range(self.n):
            if x[i].is_integer():
                
                if x[i] < 1:
                    subgrad[i] = self.get_value_fi(x[i]+1,i) - self.get_value_fi(x[i],i)
                else:
                    subgrad[i] = 1/2*(self.get_value_fi(x[i]+1,i) - self.get_value_fi(x[i]-1,i))
                    
            else:
                subgrad[i] = self.get_value_fi(np.ceil(x[i]),i) - self.get_value_fi(np.floor(x[i]),i)

        return subgrad

    def linearization(self):

        # set up linear model
        ml = model.Model(log_output=False)
        ml.parameters.timelimit = self.timeout

        # set variables
        x = ml.integer_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set auxiliary variables 
        ijk = [(i,j,k) for i in range(self.n) 
                      for j in range(self.m)
                      for k in range(self.pmax[i])]
        
        y = ml.continuous_var_dict(ijk, name = "y")

        # set up constraints
        ml.add_constraint(ml.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # add y constraints
        ml.add_constraints(y[i,j,k] >= self.prob[i][j,k]*k*self.beta[i] for (i,j,k) in ijk)
        ml.add_constraints(y[i,j,k] >= self.prob[i][j,k]*(self.beta[i]*x[i] + self.gamma[i]* (k - x[i]))
                          for (i,j,k) in ijk)

        # objective function
        ml.minimize(ml.sum(self.m*self.alpha[i]*x[i] for i in range(self.n))
                            + ml.sum(y[i,j,k] for (i,j,k) in ijk))

        ml.solve()

        return ml.objective_value, ml.solve_details.time, ml.solve_details.mip_relative_gap
    
    def linearization_relaxed(self):

        # set up linear model
        ml = model.Model(log_output=False)
        ml.parameters.timelimit = self.timeout

        # set variables
        x = ml.continuous_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set auxiliary variables 
        ijk = [(i,j,k) for i in range(self.n) 
                      for j in range(self.m)
                      for k in range(self.pmax[i])]
        
        y = ml.continuous_var_dict(ijk, name = "y")

        # set up constraints
        ml.add_constraint(ml.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # add y constraints
        ml.add_constraints(y[i,j,k] >= self.prob[i][j,k]*k*self.beta[i] for (i,j,k) in ijk)
        ml.add_constraints(y[i,j,k] >= self.prob[i][j,k]*(self.beta[i]*x[i] + self.gamma[i]* (k - x[i]))
                          for (i,j,k) in ijk)

        # objective function
        ml.minimize(ml.sum(self.m*self.alpha[i]*x[i] for i in range(self.n))
                            + ml.sum(y[i,j,k] for (i,j,k) in ijk))

        ml.solve()

        return ml.objective_value, ml.solve_details.time, ml.solve_details.mip_relative_gap


    def cutting_plane(self,iter_max: int = 200, tol: float = 1e-4):

        # set up linear model
        m = model.Model(log_output=False)
        m.parameters.timelimit = self.timeout

        # set variables
        x = m.integer_var_list(self.n, name = "x", ub = self.pmax, lb = 0)

        # set up constraints
        m.add_constraint(m.sum(self.c[i]*x[i] for i in range(self.n)) <= self.b)

        # get initial solution
        m.minimize(m.sum(self.alpha[i]*x[i] for i in range(self.n)))
        
        m.solve()
        xk = np.array(m.solution.get_value_list(x))
        
        # set upperbound
        lower = 0
        upper = lower + 1
        num_iter = 0
        time_solve = 0

        # set theta
        theta = m.continuous_var_list(self.n,name="theta")

        # objective function
        m.minimize(m.sum(theta[i] for i in range(self.n)) + self.m * m.sum(self.alpha[i]*x[i] for i in range(self.n)))

        # get oracle
        oracle = set()

        # start adding cutting planes
        while upper - lower > tol and num_iter < iter_max and time_solve < self.timeout: 

            # get gradient
            #dfx = self.get_sub_gradient(xk)


            # update the cutting plane
            #m.add_constraint(m.sum(theta[i] for i in range(self.n)) >= 
            #                 m.sum( dfx[i] * (x[i] - xk[i]) for i in range(self.n)) 
            #                 + self.get_value(xk)
            #                 )
            
            # get positive indices
            #x_positive = np.where(xk == 0)[0]

            # update more cutting planes
            m.add_constraints(theta[i] >= self.get_subgrad_i(xk[i], i)*(x[i]-xk[i])+self.get_value_fi(xk[i],i)
                              for i in range(self.n) 
                              if (i,xk[i]) not in oracle)
            

            # update oracle
            oracle = oracle.union(set([(i,xk[i]) for i in range(self.n)]))

            # solve the new model
            m.solve()
            sol = m.solution 

            # get new vector
            xk = np.array(sol.get_value_list(x))
            
            # update lower bound
            lower = m.objective_value

            # update upperbound
            upper = self.m * sum(self.alpha[i]*xk[i] for i in range(self.n)) + self.get_value(xk)

            # update steps
            num_iter += 1
            
            # update timesolve
            time_solve += m.solve_details.time
            
        
            
        optimal = 1 if upper < tol else 0
        
        return upper, m.objective_value, time_solve, num_iter, optimal



# Algorithms testing

In [171]:
data = DataGeneration(500,100,100,1)
solution = Algorithms(data)

In [ ]:
solution.cutting_plane()

In [138]:
solution.linearization()

(268570706.27343845, 200.12696886062622, 1.0)